In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime
import sklearn
import itertools
import warnings
import datetime
import os
import io
import plotly.offline as py #visualization
import plotly.graph_objs as go #visualization
import plotly.tools as tls #visualization
import plotly.figure_factory as ff #visualization
from pylab import rcParams
from typing import Dict, List, Optional, Tuple, Union
from tqdm import tqdm
from IPython.display import Image
from category_encoders import CountEncoder
from prince import FAMD,MCA,PCA
from matplotlib.colors import ListedColormap
from matplotlib import pyplot
from optbinning import OptimalBinning
from collections import Counter
from copy import deepcopy
from random import sample
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from xgboost import XGBRegressor, XGBClassifier, plot_importance
from __future__ import print_function
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn import metrics
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, label_binarize
from sklearn.decomposition import PCA as skPCA, TruncatedSVD
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, ParameterGrid, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.calibration import CalibratedClassifierCV, calibration_curve, _CalibratedClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import brier_score_loss, r2_score, precision_score, recall_score, f1_score, roc_auc_score 
from sklearn.metrics import accuracy_score, classification_report,average_precision_score,accuracy_score
from sklearn.metrics import confusion_matrix,roc_auc_score, precision_recall_curve,roc_curve, confusion_matrix
from sklearn.metrics import classification_report, silhouette_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils._testing import ignore_warnings
warnings.filterwarnings("ignore")
py.init_notebook_mode(connected=True) #visualization
%matplotlib inline
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

# Python - Datetime

In [9]:
user_data = pd.read_csv('user_data.csv',delimiter=',')
test_data = pd.read_csv('test_data.csv',delimiter=',')
data = pd.merge(user_data,test_data,how = 'left',on='user_id',)
data['last_trx_date'] = pd.to_datetime(data['last_trx_date'])
data = data.dropna()
data.head(1)

,user_id,is_premium_user,last_trx_date
0,3057801831,0,2019-02-20


In [13]:
data['last_trx_date'][0].timestamp()

1550620800.0

In [3]:
def generate_dates(df:pd.DataFrame,col:str):
    df[çol] = pd.to_datetime(df[col])
    df['unixtimestamp_seconds'] = [num.timestamp() for num in df[col]]
    df['unixtimestamp_minutes'] = df['unixtimestamp_seconds']/60
    df['unixtimestamp_hours'] = df['unixtimestamp_minutes']/60
    df['unixtimestamp_days'] = df['unixtimestamp_hours']/24    
    df['year'] = [pd.to_datetime(num).strftime("%Y") for num in df[col]]
    df['month'] = [pd.to_datetime(num).strftime("%m") for num in df[col]]
    df['name_of_month'] = [pd.to_datetime(num).strftime("%b") for num in df[col]]    
    df['day_of_month'] = [pd.to_datetime(num).strftime("%d") for num in df[col]]
    df['hour_clock'] = [pd.to_datetime(num).strftime("%H") for num in df[col]]
    df['name_of_weekday'] = [datetime.datetime.fromisoformat(str(num)).strftime("%A") for num in df[col]]
    df['number_of_weekday'] = [pd.to_datetime(num).strftime("%w") for num in df[col]]
    df['week_number_of_year'] = [pd.to_datetime(num).strftime("%W") for num in df[col]]
    df['date_days_hour'] = df['year']+df['month']+df['day_of_month']+df['hour_clock']
    df['date_days'] = df['year']+df['month']+df['day_of_month']
    df['year_month'] = df['year']+df['month']
    df['year_week_day'] = df['year']+df['week_number_of_year']+df['number_of_weekday']
    #maksimal days to churns
    df['date_20days_after'] = pd.to_datetime(df[col])+datetime.timedelta(days=20)
    df['running_full_dates'] = datetime.datetime.today()
    df['diff_days'] = np.round([num.total_seconds()/(3600*24) for num in (df['running_full_dates']-df['date_20days_after'])])
    df['running_date'] = pd.to_datetime(datetime.date.today().strftime("%Y-%m-%d"))
    return df

In [49]:
dt = pd.DataFrame(pd.date_range('2012-10-01', periods=289, freq='60min')).rename(columns={0:'unique_date'})
dfdates1 = generate_dates(dt.copy(),'unique_date')
dfdates1.head()

,unique_date,unixtimestamp_seconds,unixtimestamp_minutes,unixtimestamp_hours,unixtimestamp_days,year,month,name_of_month,day_of_month,hour_clock,name_of_weekday,number_of_weekday,week_number_of_year,year_month,year_week_day,date_20days_after,running_full_dates,diff_days,running_date
0,2012-10-01 00:00:00,1.349050e+09,22484160.0,374736.0,15614.000000,2012,10,Oct,01,00,Monday,1,40,201210,2012401,2012-10-21 00:00:00,2021-12-01 23:55:51.257616,3329.0,2021-12-01
1,2012-10-01 01:00:00,1.349053e+09,22484220.0,374737.0,15614.041667,2012,10,Oct,01,01,Monday,1,40,201210,2012401,2012-10-21 01:00:00,2021-12-01 23:55:51.257616,3329.0,2021-12-01
2,2012-10-01 02:00:00,1.349057e+09,22484280.0,374738.0,15614.083333,2012,10,Oct,01,02,Monday,1,40,201210,2012401,2012-10-21 02:00:00,2021-12-01 23:55:51.257616,3329.0,2021-12-01
3,2012-10-01 03:00:00,1.349060e+09,22484340.0,374739.0,15614.125000,2012,10,Oct,01,03,Monday,1,40,201210,2012401,2012-10-21 03:00:00,2021-12-01 23:55:51.257616,3329.0,2021-12-01
4,2012-10-01 04:00:00,1.349064e+09,22484400.0,374740.0,15614.166667,2012,10,Oct,01,04,Monday,1,40,201210,2012401,2012-10-21 04:00:00,2021-12-01 23:55:51.257616,3329.0,2021-12-01


In [4]:
dfdates = generate_dates(test_data.copy(),'last_trx_date')
dfdates.head()

,user_id,last_trx_date,unixtimestamp_seconds,unixtimestamp_minutes,unixtimestamp_hours,unixtimestamp_days,year,month,name_of_month,day_of_month,hour_clock,name_of_weekday,number_of_weekday,week_number_of_year,year_month,year_week_day,date_20days_after,running_full_dates,diff_days,running_date
0,3057937196,2019-02-20,1.550621e+09,25843680.0,430728.0,17947.0,2019,02,Feb,20,00,Wednesday,3,07,201902,2019073,2019-03-12,2021-12-01 22:53:13.379880,996.0,2021-12-01
1,3725777565,2019-02-28,1.551312e+09,25855200.0,430920.0,17955.0,2019,02,Feb,28,00,Thursday,4,08,201902,2019084,2019-03-20,2021-12-01 22:53:13.379880,988.0,2021-12-01
2,3850602392,2019-02-26,1.551139e+09,25852320.0,430872.0,17953.0,2019,02,Feb,26,00,Tuesday,2,08,201902,2019082,2019-03-18,2021-12-01 22:53:13.379880,990.0,2021-12-01
3,4218672051,2019-02-28,1.551312e+09,25855200.0,430920.0,17955.0,2019,02,Feb,28,00,Thursday,4,08,201902,2019084,2019-03-20,2021-12-01 22:53:13.379880,988.0,2021-12-01
4,4300836811,2019-01-17,1.547683e+09,25794720.0,429912.0,17913.0,2019,01,Jan,17,00,Thursday,4,02,201901,2019024,2019-02-06,2021-12-01 22:53:13.379880,1030.0,2021-12-01


In [467]:
display(datetime.date(2021, 12, 1).isoweekday())
display(datetime.date(2021, 12, 1).isoformat())
display(datetime.date(2021, 12, 1).ctime())

(2021, 48, 3)

3

'2021-12-01'

datetime.date(2021, 12, 1)

'Wed Dec  1 00:00:00 2021'

In [427]:
display(datetime.datetime(2021, 12, 1).isocalendar())
display(datetime.datetime(2021, 12, 1).isoweekday())
display(datetime.datetime(2021, 12, 1).isoformat())
display(datetime.datetime.fromisoformat(datetime.datetime(2021, 12, 1).isoformat()))
display(datetime.datetime(2021, 12, 1).ctime())

(2021, 48, 3)

3

'2021-12-01T00:00:00'

datetime.datetime(2021, 12, 1, 0, 0)

'Wed Dec  1 00:00:00 2021'

# Python - MySQL Connector

In [7]:
from getpass import getpass
import mysql.connector
import pymysql
import sqlalchemy

In [14]:
#create database
#cara 1
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=input("enter username:"),
  password=getpass("Enter Password:"),
)
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE if not exists pythonDb")

#cara 2
connection = pymysql.connect(
  host="127.0.0.1",
  user=input("enter username:"),
  password=getpass("Enter Password:"),
  database="pythonDb"    
)
mycursor1 = connection.cursor()
mycursor1.execute("CREATE DATABASE if not exists pythonDb1")

enter username:esisram
Enter Password:········
enter username:esisram
Enter Password:········


1

In [15]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('pythondb',)
('pythondb1',)
('sakila',)
('sales_distribution',)
('sys',)
('world',)


In [16]:
mycursor1.execute("SHOW DATABASES")
for x in mycursor1:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('pythondb',)
('pythondb1',)
('sakila',)
('sales_distribution',)
('sys',)
('world',)


In [17]:
mydb.close()
connection.close()

In [18]:
#directly connect to the database selected
#cara 1
mydb = pymysql.connect(
  host="127.0.0.1",
  user=input("enter username:"),
  password=getpass("Enter Password:"),
  database="pythonDb"    
)
#create table customer
mycursor = mydb.cursor()
mycursor.execute('drop table if exists customers')
mycursor.execute("""
Create table if not exists customers (
name varchar(255),
address varchar(255),
test integer
)""")

#cara 2
connection = mysql.connector.connect(
  host="127.0.0.1",
  user=input("enter username:"),
  password=getpass("Enter Password:"),
  database="pythonDb1"    
)
#create table customer
mycursor1 = connection.cursor(buffered=True)
mycursor1.execute('drop table if exists customers')
mycursor1.execute("""
Create table if not exists customers (
name varchar(255),
address varchar(255),
test integer
)""")

enter username:esisram
Enter Password:········
enter username:esisram
Enter Password:········


In [19]:
mycursor.execute("show tables")
for x in mycursor:
    print(x)

('customers',)


In [20]:
mycursor.execute("alter TABLE customers add column id int auto_increment primary key")

0

In [21]:
mycursor.execute('show create table customers')
for x in mycursor:
    print(x)

('customers', 'CREATE TABLE `customers` (\n  `name` varchar(255) DEFAULT NULL,\n  `address` varchar(255) DEFAULT NULL,\n  `test` int DEFAULT NULL,\n  `id` int NOT NULL AUTO_INCREMENT,\n  PRIMARY KEY (`id`)\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci')


In [22]:
mycursor.execute("alter TABLE customers drop test")

0

Insert Into

In [23]:
sql = 'insert into customers (name,address) values("esis","krisan 21")'
mycursor.execute(sql)
mydb.commit()
print(mycursor.rowcount,'record inserted')

1 record inserted


In [24]:
sql = 'insert into customers (name,address) values (%s,%s)'
val = ('astro','aster 21')
mycursor.execute(sql,val)
mydb.commit()
print(mycursor.rowcount,'record inserted')

1 record inserted


In [25]:
sql = "insert into customers (name,address) values ('aris','arrahman 21'),('asri','france 21'),('papah','japang 21')"
mycursor.execute(sql)
mydb.commit()
print(mycursor.rowcount,'record inserted')

3 record inserted


In [26]:
sql = 'insert into customers (name,address) values (%s,%s)'
val = [('dera','kamboja 21')]
mycursor.executemany(sql,val)
mydb.commit()
print(mycursor.rowcount,'record inserted')

1 record inserted


select

In [27]:
mycursor.execute("create table if not exists customer_copy as select * from customers")

6

In [28]:
mycursor.execute('delete from customer_copy where address like "kam%"')
mydb.commit()
print(mycursor.rowcount)

1


In [29]:
sql = 'delete from customers where address = %s'
adr = ('aster 21',)
mycursor.execute(sql,adr)
mydb.commit()
print(mycursor.rowcount)

1


In [30]:
sql = 'update customers set address = %s where address = %s'
adr = [('krisan 22', 'krisan 21'),('aster 22', 'aster 21')]
mycursor.executemany(sql,adr)
mydb.commit()
print(mycursor.rowcount)

1


In [31]:
mycursor.execute("select * from customer_copy")
myresult = mycursor.fetchall()
print(myresult)

(('esis', 'krisan 21', 1), ('astro', 'aster 21', 2), ('aris', 'arrahman 21', 3), ('asri', 'france 21', 4), ('papah', 'japang 21', 5))


In [32]:
df = pd.DataFrame(myresult).rename(columns={0:'name',1:'address',2:'ID'})
df['running_date'] = datetime.date.today()
df['running_date1'] = datetime.datetime.now()
df

,name,address,ID,running_date,running_date1
0,esis,krisan 21,1,2021-12-01,2021-12-01 22:54:30.396079
1,astro,aster 21,2,2021-12-01,2021-12-01 22:54:30.396079
2,aris,arrahman 21,3,2021-12-01,2021-12-01 22:54:30.396079
3,asri,france 21,4,2021-12-01,2021-12-01 22:54:30.396079
4,papah,japang 21,5,2021-12-01,2021-12-01 22:54:30.396079


In [33]:
mydb.close()
connection.close()

In [35]:
# insert pandas dataframe into MySQL
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{localhost}/{db}"
                       .format(user=input("enter username:"),
                               localhost="127.0.0.1",
                               pw=getpass("Enter Password:"),
                               db="pythonDb"))
df.to_sql(name='customer_copy',con=engine, if_exists='replace')

enter username:esisram
Enter Password:········


In [ ]:
# insert pandas dataframe into MySQL
from sqlalchemy import create_engine
engine = create_engine("mysql+mysqlconnector://{user}:{pw}@{localhost}/{db}"
                       .format(user=input("enter username:"),
                               localhost="127.0.0.1",
                               pw=getpass("Enter Password:"),
                               db="pythonDb"))
df.to_sql(name='customer_copy',con=engine, if_exists='replace')

# python - monggodb

In [37]:
import pymongo

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]